In [85]:
import pandas as pd
from numpy import dot
from numpy.linalg import norm
import numpy as np
from numba import jit, njit

In [86]:
@jit
def cosine_sim(a, b): 
    return dot(a, b)/(norm(a)*norm(b))
[cosine_sim(np.random.normal(size=(800,)),np.random.normal(size=(800,))) for _ in range(20000)];

In [69]:
bertopic_emb = pd.read_csv('../results/embeddings/bertopic_processed_data_2022-11-22_23-14-24_test_embeddings_model_pretrained.csv')
lda_emb = pd.read_csv('../results/embeddings/LDA_test_2022-11-23_17-47-24_embeddings_model_pretrained.csv')

In [70]:
concepts_emb_pretrained = pd.read_csv('../results/embeddings/concepts_emb_pretrained_model.csv')

In [71]:
concepts_emb_pretrained_raw = concepts_emb_pretrained.values[:, :-1].astype(np.float32)

In [89]:
def return_embedded_word(row):
    embedding = row.values[5:].astype(np.float32).flatten()
    cosine_similarities = np.apply_along_axis(lambda x: cosine_sim(embedding, x), axis=1, arr=concepts_emb_pretrained_raw)
    return concepts_emb_pretrained.iloc[cosine_similarities.argmax()]['concept_name']

In [90]:
import tqdm
tagged_words = []
for i, row in tqdm.tqdm(bertopic_emb.iterrows(), total=len(bertopic_emb)):
    tagged_word = return_embedded_word(row)
    tagged_words.append(tagged_word)
bertopic_emb['tagged_word'] = tagged_words

100%|██████████████████████████████████████████████████████████████████████████████| 8790/8790 [12:01<00:00, 12.19it/s]


In [92]:
def collect_list(x): 
    result= {}
    result['tagged_words'] = list(x['tagged_word'])
    return pd.Series(result, index=['tagged_words'])
bertopic_emb.groupby('PMID').apply(collect_list).reset_index().to_csv('../results/tagged_bertopic_bert_pretrained.csv', index=False)